In [1]:
import os, glob, json, pickle
from openeye import *

In [2]:
#Extract the mol2 files
os.system('tar -xvzf ../mol2files_gaff.tar.gz')

0

In [3]:
#Load in our input files
mol2files = sorted(glob.glob('mol2files_gaff/*.mol2'))
dbfile = open('../database.json')
db = json.load(dbfile)

In [4]:
#Initialize our outputfile
outfname = 'freesolv.oeb.gz'
ofs = oemolostream()
if not ofs.open(outfname):
    OEThrow.Fatal("Unable to open %s for writing" % outfname)

In [5]:
#Loop over mol2 files
omol = OEGraphMol()    
for mol2 in mol2files:
    fname = mol2.split('/')[-1]
    label = fname.split('.mol2')[0]

    with oemolistream(mol2) as ifs:
        for imol in ifs.GetOEGraphMols():
            # Add label tag
            imol.SetData(OEGetTag('label'), label)
            
            # Attach each field as a new tag
            for k,v in sorted(db[label].items()):
                #print(label,k,v)
                imol.SetData(OEGetTag(k), v)
            
            # Encode entire dict for each mol
            data = json.dumps(db[label])
            imol.SetData(OEGetTag('database'), data)
            
            OEWriteMolecule(ofs, imol)
            
ofs.close()

In [8]:
# Check that our database is unchanged
with oemolistream(outfname) as ifs:
    for imol in ifs.GetOEGraphMols():
        #print(imol.GetData().keys())
        label = (imol.GetData('label'))
        dbfile = imol.GetData('database')
        
        reconst_db = json.loads(dbfile)
        assert db[label] == reconst_db